In [21]:
# File Directory 
import glob
import os
from os.path import isdir, join
from pathlib import Path

# Math
import numpy as np
from scipy.fftpack import fft
from scipy import signal
import librosa

# Dimension Reduction
from sklearn.decomposition import PCA

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
import librosa.display

# Data Pre-processing
import pandas as pd
from sklearn.model_selection import KFold

# Deep Learning
import tensorflow as tf
from tensorflow.keras import Input, layers, backend as K
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation
from tensorflow.keras.layers import BatchNormalization, Flatten, multiply
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import cv2
import re

# Configuration
#py.init_notebook_mode(connected=True)

%matplotlib inline

In [ ]:
samples = np.load("samples.npy")
labels = np.load("labels.npy")

In [ ]:
def show(spectrogram):
    plt.imshow(spectrogram)
    plt.show()

In [ ]:
#Make spectrograms
def make_spectrogram(y,sr):
    return np.array(librosa.feature.melspectrogram(y=y, sr=sr))

In [32]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

number_of_epochs = 50
batch_size = 32
optimizer = Adam(lr=0.001, decay=0.001 / 100)
input_tensor = Input(shape=(128, 64, 1))
window = (4, 4)

""" Step 2: Create the input and hidden layers """

# First Layer
x = Conv2D(32, window, padding="same", activation="relu")(input_tensor)
x = BatchNormalization(axis=-1)(x)
x = MaxPooling2D(pool_size=(2, 1))(x)
x = Dropout(0.25)(x)

# Second Layer: (CONV => RELU) * 2 => POOL
x = Conv2D(64, window, activation="relu", padding="same")(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(64, window, activation="relu", padding="same")(x)
x = BatchNormalization(axis=-1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

# Third Layer: (CONV => RELU) * 2 => POOL
x = Conv2D(128, window, activation="relu", padding="same")(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(128, window, activation="relu", padding="same")(x)
x = BatchNormalization(axis=-1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

# Fourth Layer: (CONV => RELU) * 2 => POOL
x = Conv2D(256, window, activation="relu", padding="same")(x)
x = BatchNormalization(axis=-1)(x)
x = Conv2D(256, window, activation="relu", padding="same")(x)
x = BatchNormalization(axis=-1)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)


""" Step 3: Flatten the layers """

x = Flatten()(x)

""" Step 4: Fully-connect the layers """

x = Dense(1024, activation="relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Increasing dropout here to prevent overfitting

#Attention layer
attention_probs = Dense(1024, activation='softmax', name='attention_vec')(x)
attention_probs = Dropout(0.5)(attention_probs)
attention_mul = multiply([x, attention_probs], name='attention_mul')
attention_mul = Dense(256)(attention_mul)

'''x = Dense(256, activation="relu")(attention_mul)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Increasing dropout here to prevent overfitting
'''

output_tensor = layers.Dense(2, activation="softmax")(attention_mul)

""" Step 5: Compile the model """
model = tf.keras.Model(input_tensor, output_tensor)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=[auc, "accuracy"])


In [33]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 128, 64, 1)   0                                            
__________________________________________________________________________________________________
conv2d_85 (Conv2D)              (None, 128, 64, 32)  544         input_14[0][0]                   
__________________________________________________________________________________________________
batch_normalization_v1_100 (Bat (None, 128, 64, 32)  128         conv2d_85[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_48 (MaxPooling2D) (None, 64, 64, 32)   0           batch_normalization_v1_100[0][0] 
__________________________________________________________________________________________________
dropout_67

In [ ]:
model.load_weights("model_2d_4l.h5")

In [ ]:
from sklearn.preprocessing import LabelBinarizer

In [ ]:
labels = np.load("labels.npy")

In [ ]:
urban = ['air_conditioner', 'car_horn', 'children_playing', 'dog_bark', 'other',
       'drilling', 'engine_idling', 'jackhammer', 'siren', 'street_music']
for i in range(len(labels)):
    if labels[i] in urban:
        labels[i] = 'other'


lb = LabelBinarizer()
labels = lb.fit_transform(labels)

## Validation

In [ ]:
def to_spec(data):
    a = make_spectrogram(data,22050)
    plt.interactive(False)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    librosa.display.specshow(librosa.power_to_db(a, ref=np.max))
    plt.savefig('temp.png', dpi=400, bbox_inches='tight',pad_inches=0)
    im = cv2.imread('temp.png')
    im = cv2.resize(im, (192, 192))

    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    return im

In [ ]:
def show_wav(data):
    plt.figure(figsize=(12, 4))
    librosa.display.waveplot(data, sr = 22050)

In [ ]:
mypath = "C:\\Users\\hosle\\Downloads\\shotgun-old_school-RA_The_Sun_God-1129942741.wav"
#mypath = "C:\\Users\\hosle\\Documents\\_REU2019\\extra\\260600.wav"

fire, sr = librosa.load(mypath, res_type='kaiser_fast')

In [ ]:
validation = []
if (fire.size <= 44100):
    sample_slice = np.zeros(44100)
    sample_slice[0:fire.size] = fire
    validation.append(sample_slice)

for i in range(0, fire.size - 44100, 44100):
    sample_slice = fire[i : i + 44100]
    validation.append(sample_slice)
    
validation = np.array(validation)

val_im = []
for i in range(len(validation)):
    val_im.append(to_spec(validation[i]))
val_im = np.array(val_im).astype('float32')
val_im /= 255

val_pred = model.predict(val_im)
val_pred1 = lb.inverse_transform(val_pred)
print(len(val_pred1))
print("gun_shot", np.argwhere(val_pred1 == 'gun_shot').T)
print(val_pred1)

In [ ]:
i = 0
print(np.round(val_pred[i, :], 3))
print(val_pred1[i])
show_wav(validation [i])

ipd.Audio(validation [i], rate=22050)

In [6]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

point = Point(0.5, 0.5)
polygon = Polygon([(0, 0), (0, 1), (1, 1), (1, 0)])
print(polygon.contains(point))

True
